In [29]:
import pandas as pd
import numpy as np
import yfinance as yf

# Found stationary series: 17.73*BTC-USD + 2.09*BNB-USD + -18.82*WBTC-USD

# Found stationary series: -380.18*BTC-USD + -41.55*SOL-USD + 422.73*WBTC-USD

# Found stationary series: 26.81*BTC-USD + 4.03*XRP-USD + -29.83*WBTC-USD

# Found stationary series: 0.09*BTC-USD + 1.00*USDC-USD + -0.09*WBTC-USD

# Found stationary series: 31.37*BTC-USD + 4.09*ADA-USD + -34.46*WBTC-USD

# Found stationary series: 57.62*BTC-USD + 0.06*AVAX-USD + -56.68*WBTC-USD

# Found stationary series: 28.19*BTC-USD + 2.11*DOGE-USD + -29.30*WBTC-USD

# Found stationary series: 28.89*BTC-USD + 0.77*TRX-USD + -28.66*WBTC-USD

Downloading data for required combinations

In [30]:
btc_data = yf.download('BTC-USD', start='2022-01-01', end='2023-11-01')['Close']
bnb_data = yf.download('BNB-USD', start='2022-01-01', end='2023-11-01')['Close']
wbtc_data = yf.download('WBTC-USD', start='2022-01-01', end='2023-11-01')['Close']
sol_data = yf.download('SOL-USD', start='2022-01-01', end='2023-11-01')['Close']
xrp_data = yf.download('XRP-USD', start='2022-01-01', end='2023-11-01')['Close']
usdc_data = yf.download('USDC-USD', start='2022-01-01', end='2023-11-01')['Close']
ada_data = yf.download('ADA-USD', start='2022-01-01', end='2023-11-01')['Close']
avax_data = yf.download('AVAX-USD', start='2022-01-01', end='2023-11-01')['Close']
doge_data = yf.download('DOGE-USD', start='2022-01-01', end='2023-11-01')['Close']
trx_data = yf.download('TRX-USD', start='2022-01-01', end='2023-11-01')['Close']

combinations = [
    {'weights': [17.73, 2.09, -18.82], 'assets': ['BTC-USD', 'BNB-USD', 'WBTC-USD']},
    {'weights': [-380.18, -41.55, 422.73], 'assets': ['BTC-USD', 'SOL-USD', 'WBTC-USD']},
    {'weights': [26.81, 4.03, -29.83], 'assets': ['BTC-USD', 'XRP-USD', 'WBTC-USD']},
    {'weights': [0.09, 1.00, -0.09], 'assets': ['BTC-USD', 'USDC-USD', 'WBTC-USD']},
    {'weights': [31.37, 4.09, -34.46], 'assets': ['BTC-USD', 'ADA-USD', 'WBTC-USD']},
    {'weights': [57.62, 0.06, -56.68], 'assets': ['BTC-USD', 'AVAX-USD', 'WBTC-USD']},
    {'weights': [28.19, 2.11, -29.30], 'assets': ['BTC-USD', 'DOGE-USD', 'WBTC-USD']},
    {'weights': [28.89, 0.77, -28.66], 'assets': ['BTC-USD', 'TRX-USD', 'WBTC-USD']}
]

df = pd.DataFrame(index=btc_data.index)  

for idx, combination in enumerate(combinations):
    weights = combination['weights']
    assets = combination['assets']
    
    combination_series = sum(weight * yf.download(asset, start='2022-01-01', end='2023-11-01')['Close'] for weight, asset in zip(weights, assets))
    
    df[f'Combination_{idx+1}'] = combination_series.values


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

    Data Cleaning:
        df = df.dropna(): Removes any incomplete or missing data from the dataset, ensuring a clean dataset for analysis.

    Loop Over Strategies:
        for idx in range(len(combinations)):: Iterates through different combinations of financial assets, where each combination has specific weights assigned to different assets.

    Bollinger Bands Strategy Function:
        def bollinger_bands_strategy(series):: Defines a function that takes a time series (historical prices of a financial asset) and applies a Bollinger Bands trading strategy.

    Calculating Moving Averages and Bands:
        Calculates the moving average and standard deviation of the asset's prices, which are used to determine upper and lower Bollinger Bands.

    Position Sizing and Initialization:
        Sets a constant value representing the size of the trading position.
        Initializes a series to store the trading positions.

    Generating Trading Signals:
        Based on Bollinger Bands, generates signals to buy or sell the asset.

    Applying Strategy to Asset Prices:
        Applies the Bollinger Bands strategy to the historical prices of each financial asset within a specific combination.

    Storing Trading Positions in DataFrame:
        Creates a new column in the DataFrame to store the trading positions based on the applied strategy.

    Data Cleaning After Strategy Application:
        df = df.dropna(): Removes any rows with missing values introduced during the strategy application.

In [31]:
df = df.dropna()
for idx in range(len(combinations)):
    def bollinger_bands_strategy(series):
        sma = series.rolling(window=30).mean()
        sd = series.rolling(window=30).std()

        upper_band = sma + 2 * sd
        lower_band = sma - 2 * sd

        position_size = 0.5

        position = pd.Series(0, index=series.index)

        position[series > upper_band] = -position_size
        position[series > (sma + 4 * sd)] = -position_size

        position[series < lower_band] = position_size
        position[series < (sma - 4 * sd)] = position_size

        return position

    df[f'Combination_{idx+1}_Position'] = bollinger_bands_strategy(df[f'Combination_{idx+1}'])

df = df.dropna()

C:\Users\Mohammad\AppData\Local\Temp\ipykernel_18676\111838145.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  position[series > upper_band] = -position_size
C:\Users\Mohammad\AppData\Local\Temp\ipykernel_18676\111838145.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  position[series > upper_band] = -position_size
C:\Users\Mohammad\AppData\Local\Temp\ipykernel_18676\111838145.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  position[series > upper_band] = -position_size
C:\Users\M

    Calculate Daily Returns:
        For each combination, it calculates the daily returns based on the previously determined trading positions. The returns are computed by multiplying the trading position (buy/sell signal) with the percentage change in the combination's value.

    Calculate Equity:
        It calculates the equity for each combination, representing the cumulative growth of the investment over time. This is done by taking the cumulative product of (1 + daily returns).

    Calculate Sharp Ratio:
        It calculates the Sharp ratio for each combination using the computed returns. The Sharp ratio is a measure of risk-adjusted return, indicating how well the investment has performed relative to its risk.

    Print Results:
        For each combination, it prints the calculated Sharp ratio and the equity values over time.

In [32]:
for idx in range(len(combinations)):
    df[f'Combination_{idx+1}_Returns'] = df[f'Combination_{idx+1}_Position'] * df[f'Combination_{idx+1}'].pct_change()

    df[f'Combination_{idx+1}_Equity'] = (1 + df[f'Combination_{idx+1}_Returns']).cumprod()

    def sharp_ratio(returns):
        portfolio_profit = np.average(returns)
        risk_free_profit = 0.04 / 365
        std = np.std(returns)
        sharp_ratio = (portfolio_profit - risk_free_profit) / std * np.sqrt(365)
        return sharp_ratio

    sharp = sharp_ratio(df[f'Combination_{idx+1}_Returns'].dropna())
    print(f"Sharp Ratio for Combination {idx+1}: {sharp}")

    print(f"Equity for Combination {idx+1}:\n{df[f'Combination_{idx+1}_Equity']}\n")



Sharp Ratio for Combination 1: 2.438144315689429
Equity for Combination 1:
Date
2022-01-01         NaN
2022-01-02    1.000000
2022-01-03    1.000000
2022-01-04    1.000000
2022-01-05    1.000000
                ...   
2023-10-27    6.659882
2023-10-28    6.659882
2023-10-29    6.777550
2023-10-30    6.777550
2023-10-31    6.777550
Name: Combination_1_Equity, Length: 669, dtype: float64

Sharp Ratio for Combination 2: -3.5174800072635137
Equity for Combination 2:
Date
2022-01-01         NaN
2022-01-02    1.000000
2022-01-03    1.000000
2022-01-04    1.000000
2022-01-05    1.000000
                ...   
2023-10-27    0.193535
2023-10-28    0.193535
2023-10-29    0.191051
2023-10-30    0.191051
2023-10-31    0.191051
Name: Combination_2_Equity, Length: 669, dtype: float64

Sharp Ratio for Combination 3: 3.23405289311786
Equity for Combination 3:
Date
2022-01-01         NaN
2022-01-02    1.000000
2022-01-03    1.000000
2022-01-04    1.000000
2022-01-05    1.000000
                ...   
2